# Metrics
1) `text match` but `explanation !match` = -1
2) `text match` and `explanation match` = +1
3) `text !match` and `explanation match` = -1
4) `text !match` and `explanation !match` = -1

In [1]:
import os
import json
import tqdm
import threading
from concurrent.futures import ThreadPoolExecutor
import os
import contextlib

In [2]:
from modules.prompts import COT, ZERO_SHOT_PROMPT, FEW_SHOT_PROMPT
from modules import utils
from modules.models import Model, GeminiModel, SelfVerificationModel
from modules.dataset import Dataset, MiniEvalDataset
from modules import explanation_match as em
from modules import evaluate as eval

c:\Users\Noel\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
API_KEYS = [
]

You retrieve elements in each dataset like this:

In [4]:
dataset = MiniEvalDataset()
display(dataset[0]["answers"], dataset[0]["documents"])


[{'file_name': 'ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENTAGREEMENT.txt',
  'perturbation': [{'type': 'Ambiguities - In Text Contradiction',
    'original_text': 'A. CONSULTANT\'S "ENDORSEMENT" means the right to use the CONSULTANT\'S name, fame, nickname, autograph, voice, facsimile, signature, photograph, likeness, and image in connection with the marketing, advertising, promotion and sale of ADAMS GOLF\'S PRODUCT.',
    'changed_text': 'A. CONSULTANT\'S "ENDORSEMENT" means the right to use the CONSULTANT\'S name solely for marketing materials directly created by ADAMS GOLF. This excludes use of likeness or image for promotional events unless specifically agreed upon in writing.',
    'explanation': 'The original definition of "ENDORSEMENT" is broad, including name, likeness, and image. The modified definition restricts the endorsement to the use of name only for marketing materials, contradicting the broad definition of endorsement in the original clause. This introduces ambiguity

'REDACTED COPY CONFIDENTIAL TREATMENT REQUESTED CONFIDENTIAL PORTIONS OF THIS DOCUMENT HAVE BEEN REDACTED AND HAVE BEEN SEPARATELY FILED WITH THE COMMISSION 1 ENDORSEMENT AGREEMENT This Agreement is entered into on January 13, 2005 between professional golfer, TOM WATSON, (hereinafter referred to as "CONSULTANT") and ADAMS GOLF, LTD. (hereinafter referred to as "ADAMS GOLF"). WITNESSETH WHEREAS, ADAMS GOLF desires to obtain the right to use the name, likeness and ENDORSEMENT of CONSULTANT in connection with the advertisement and promotion of ADAMS GOLF\'S PRODUCT; NOW THEREFORE, in consideration of the mutual covenants contained herein and other good and valuable consideration, the receipt and sufficiency of which is hereby acknowledged, the parties agree as follows: CONTRACT PERIOD 1. TERM OF CONTRACT The Term of this Agreement shall be for a period of [* ****] years and [*****] months commencing the 1st day of September 2004 and terminating the [*****] day of [*****]. 2. DEFINITIONS 

**You check the length like this:**

In [5]:
# len(dataset)
# print(dataset[5]["file_name"])

**To maintain the base file name, removing `modified_` or `perturbed_`**

In [6]:
# dataset = MiniEvalDataset()
# dataset.clean_filenames()

### Implementation of `generate_responses`

In [7]:
def generate_responses(model, dataset, prompt: str, output_dir, num_responses: int = 1):
    try:
        for sample in tqdm.tqdm(dataset, desc="Processing samples"):
            # print(sample)
            # Prepare base directory and document text
            base_name = sample["file_name"]
            document_with_tags = sample["documents"]
            document_with_tags_removed = sample["documents"].replace("<*$p$*>", "") 
            ground_truth = sample["answers"][0]["perturbation"]

            for i in range(num_responses):
                # Construct output path: outputs/self_consistency/<subdir>/<filename>_i.json
                subdir = os.path.join(output_dir, "self_consistency", os.path.dirname(base_name))
                os.makedirs(subdir, exist_ok=True)
                output_path = os.path.join(subdir, os.path.basename(base_name) + f"_{i}.json")

                # Skip if file already exists
                if os.path.exists(output_path):
                    continue

                # Generate model response
                model_response = model.generate(
                    prompt.replace("[DOCUMENT]", document_with_tags_removed)
                    # prompt.replace("[DOCUMENT]", document_with_tags)
                )
                parsed_response = utils.clean_and_parse_model_response(model_response)

                if parsed_response:
                    updated_predictions = utils.add_section_identified_flag(parsed_response, ground_truth)
                    with open(output_path, "w", encoding="utf-8") as f:
                        json.dump(updated_predictions, f, indent=4)
    except Exception as e:
        print(f"❌ Error in generate_responses: {e}")

In [8]:
def run(
    model: Model,
    dataset: Dataset,
    prompt: str,
    responses_dir: str,
    num_responses: int,
    evaluation_model: Model = None
):
    """
    Runs the evaluation process.
    :param model: The model to generate responses.
    :param dataset: The dataset to evaluate.
    :param prompt: The prompt to use for generating responses.
    :param responses_dir: Directory to save the responses.
    :param num_responses: The number of responses to collect per document (for self-consistency)
    :param evaluation_model: Model for evaluating model responses.
    """
    generate_responses(model, dataset, prompt, responses_dir, num_responses)
    # explanation_match(evaluation_model, dataset, responses_dir)
    em.explanation_match_sbert(dataset, responses_dir, threshold=0.5)
    return eval.evaluate_scoring(responses_dir)

In [9]:
runs = [
    {
        "name": "zero-shot",
        "model": GeminiModel(API_KEYS),
        "dataset": MiniEvalDataset(),
        "prompt": ZERO_SHOT_PROMPT,
        "responses_dir": utils.correct_path_name("mini-eval/responses_v2/zero-shot/"),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
    {
        "name": "zero-shot-cot",
        "model": GeminiModel(API_KEYS),
        "dataset": MiniEvalDataset(),
        "prompt": ZERO_SHOT_PROMPT + COT,
        "responses_dir": utils.correct_path_name("mini-eval/responses_v2/zero-shot-cot/"),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
    {
        "name": "zero-shot-self-verification",
        "model": SelfVerificationModel(GeminiModel(API_KEYS)),
        "dataset": MiniEvalDataset(),
        "prompt": ZERO_SHOT_PROMPT,
        "responses_dir": utils.correct_path_name(
            "mini-eval/responses_v2/zero-shot-self-verification/"
        ),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
    {
        "name": "zero-shot-self-verification-cot",
        "model": SelfVerificationModel(GeminiModel(API_KEYS)),
        "dataset": MiniEvalDataset(),
        "prompt": ZERO_SHOT_PROMPT + COT,
        "responses_dir": utils.correct_path_name(
            "mini-eval/responses_v2/zero-shot-self-verification-cot/"
        ),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
    {
        "name": "few-shot",
        "model": GeminiModel(API_KEYS),
        "dataset": MiniEvalDataset(),
        "prompt": FEW_SHOT_PROMPT,
        "responses_dir": utils.correct_path_name("mini-eval/responses_v2/few-shot/"),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
    {
        "name": "few-shot-cot",
        "model": GeminiModel(API_KEYS),
        "dataset": MiniEvalDataset(),
        "prompt": FEW_SHOT_PROMPT + COT,
        "responses_dir": utils.correct_path_name("mini-eval/responses_v2/few-shot-cot/"),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
    {
        "name": "few-shot-self-verification",
        "model": SelfVerificationModel(GeminiModel(API_KEYS)),
        "dataset": MiniEvalDataset(),
        "prompt": FEW_SHOT_PROMPT,
        "responses_dir": utils.correct_path_name(
            "mini-eval/responses_v2/few-shot-self-verification/"
        ),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
    {
        "name": "few-shot-self-verification-cot",
        "model": SelfVerificationModel(GeminiModel(API_KEYS)),
        "dataset": MiniEvalDataset(),
        "prompt": FEW_SHOT_PROMPT + COT,
        "responses_dir": utils.correct_path_name(
            "mini-eval/responses_v2/few-shot-self-verification-cot/"
        ),
        "num_responses": 1,
        "evaluation_model": GeminiModel(API_KEYS),
    },
]

In [10]:
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, "w") as fnull:
        with contextlib.redirect_stdout(fnull), contextlib.redirect_stderr(fnull):
            yield


# Semaphore to limit the number of concurrent threads to the number of API keys
api_key_semaphore = threading.Semaphore(len(API_KEYS))

run_results = {}


def run_with_semaphore(run_config):
    """
    Wrapper function to run a task while respecting the semaphore.
    """
    with api_key_semaphore:
        run_results[run_config["name"]] = run(
            model=run_config["model"],
            dataset=run_config["dataset"],
            prompt=run_config["prompt"],
            responses_dir=run_config["responses_dir"],
            num_responses=run_config["num_responses"],
            evaluation_model=run_config["evaluation_model"],
        )


with ThreadPoolExecutor(max_workers=len(API_KEYS)) as executor:
    for run_config in runs:
        executor.submit(run_with_semaphore, run_config)

print("✅ DONE")

Processing samples:   0%|          | 0/30 [00:00<?, ?it/s]




















Processing samples:  20%|██        | 6/30 [00:00<00:00, 55.77it/s]




















Processing samples:  40%|████      | 12/30 [00:00<00:00, 44.22it/s]




















Processing samples:  60%|██████    | 18/30 [00:00<00:00, 46.28it/s]




















Processing samples:  77%|███████▋  | 23/30 [00:00<00:00, 42.49it/s]





















Processing samples:  93%|█████████▎| 28/30 [00:00<00:00, 40.04it/s]

























Processing samples: 100%|██████████| 30/30 [00:00<00:00, 35.89it/s]





Processing samples: 100%|██████████| 30/30 [00:00<00:00, 34.54it/s]






Processing samples: 100%|██████████| 30/30 [00:00<00:00, 33.05it/s]




Processing samples: 100%|██████████| 30/30 [00:00<00:00, 33.71it/s]





Processing samples: 100%|██████████| 30/30 [00:00<00:00, 33.40it/s]






Processing samples: 100%|██████████| 30/30 [00:00<00:00, 32.90it/s]




Processing samples: 100%|██████████| 3

✅ Model loaded successfully!
✅ Model loaded successfully!


Evaluating explanations (SBERT):   0%|          | 0/30 [00:00<?, ?it/s]

✅ Model loaded successfully!




Evaluating explanations (SBERT):   3%|▎         | 1/30 [00:00<00:05,  5.78it/s]

...

⚠️ Skipped (no update needed): \\?\c:\Users\Noel\Documents\Projects\School\CSE 576\Legal-Document-Discrepancy-Benchmark-Dataset\mini-eval\responses_v2\zero-shot-self-verification\self_consistency\structural_flaws_inText\Freecook_20180605_S-1_EX-10.3_11233807_EX-10.3_HostingAgreement.txt_0.json
⚠️ No ground truth explanations for: unperturbed\2ThemartComInc_19990826_10-12G_EX-10.10_6700288_EX-10.10_Co-Branding Agreement_ Agency Agreement
⚠️ No ground truth explanations for: unperturbed\ABILITYINC_06_15_2020-EX-4.25-SERVICES AGREEMENT
⚠️ No ground truth explanations for: unperturbed\ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-EX-10.13-JOINT VENTURE AGREEMENT
⚠️ No ground truth explanations for: unperturbed\ACCURAYINC_09_01_2010-EX-10.31-DISTRIBUTOR AGREEMENT
⚠️ No ground truth explanations for: unperturbed\ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT AGREEMENT









Evaluating explanations (SBERT): 100%|██████████| 30/30 [00:05<00:00,  5.67it/s]










📁 Directory: ambiguity_inText
Text Match (any): 7 / 15
  ├─ v1 (changed_text): 7 / 15
  └─ v2 (contradicted_text): 2 / 15
Explanation Match: 2 / 15
Text + Explanation Match: 2 / 15

📁 Directory: inconsistencies_inText
Text Match (any): 12 / 22
  ├─ v1 (changed_text): 11 / 22
  └─ v2 (contradicted_text): 11 / 22
Explanation Match: 4 / 22
Text + Explanation Match: 2 / 22

📁 Directory: misaligned_terminalogy_inText
Text Match (any): 6 / 18
  ├─ v1 (changed_text): 6 / 18
  └─ v2 (contradicted_text): 0 / 18
Explanation Match: 1 / 18
Text + Explanation Match: 1 / 18

📁 Directory: omissions_inText
Text Match (any): 8 / 18
  ├─ v1 (changed_text): 3 / 18
  └─ v2 (contradicted_text): 5 / 18
Explanation Match: 0 / 18
Text + Explanation Match: 0 / 18

📁 Directory: structural_flaws_inText
Text Match (any): 8 / 19
  ├─ v1 (changed_text): 8 / 19
  └─ v2 (contradicted_text): 5 / 19
Explanation Match: 5 / 19
Text + Explanation Match: 3 / 19
⚠️ Skipped (no update needed): \\?\c:\Users\Noel\Documents\Pr








Evaluating explanations (SBERT): 100%|██████████| 30/30 [00:05<00:00,  5.42it/s]


⚠️ Skipped (no update needed): \\?\c:\Users\Noel\Documents\Projects\School\CSE 576\Legal-Document-Discrepancy-Benchmark-Dataset\mini-eval\responses_v2\few-shot\self_consistency\structural_flaws_inText\Freecook_20180605_S-1_EX-10.3_11233807_EX-10.3_HostingAgreement.txt_0.json
⚠️ No ground truth explanations for: unperturbed\2ThemartComInc_19990826_10-12G_EX-10.10_6700288_EX-10.10_Co-Branding Agreement_ Agency Agreement
⚠️ No ground truth explanations for: unperturbed\ABILITYINC_06_15_2020-EX-4.25-SERVICES AGREEMENT
⚠️ Skipped (no update needed): \\?\c:\Users\Noel\Documents\Projects\School\CSE 576\Legal-Document-Discrepancy-Benchmark-Dataset\mini-eval\responses_v2\few-shot-self-verification-cot\self_consistency\structural_flaws_inText\Freecook_20180605_S-1_EX-10.3_11233807_EX-10.3_HostingAgreement.txt_0.json
⚠️ No ground truth explanations for: unperturbed\ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-EX-10.13-JOINT VENTURE AGREEMENT
⚠️ No ground truth explanations for: unperturbed\ACCUR

### **Analysis**

In [11]:
import pandas as pd

df = pd.DataFrame.from_dict(run_results, orient="index")
df

,ambiguity_inText,inconsistencies_inText,misaligned_terminalogy_inText,omissions_inText,structural_flaws_inText
few-shot-self-verification,"{'text_matches': 8, 'text_match_v1': 7, 'text_...","{'text_matches': 10, 'text_match_v1': 9, 'text...","{'text_matches': 8, 'text_match_v1': 5, 'text_...","{'text_matches': 6, 'text_match_v1': 5, 'text_...","{'text_matches': 11, 'text_match_v1': 11, 'tex..."
zero-shot,"{'text_matches': 8, 'text_match_v1': 7, 'text_...","{'text_matches': 12, 'text_match_v1': 11, 'tex...","{'text_matches': 11, 'text_match_v1': 10, 'tex...","{'text_matches': 7, 'text_match_v1': 5, 'text_...","{'text_matches': 10, 'text_match_v1': 10, 'tex..."
zero-shot-self-verification-cot,"{'text_matches': 8, 'text_match_v1': 8, 'text_...","{'text_matches': 8, 'text_match_v1': 5, 'text_...","{'text_matches': 10, 'text_match_v1': 8, 'text...","{'text_matches': 10, 'text_match_v1': 8, 'text...","{'text_matches': 10, 'text_match_v1': 9, 'text..."
few-shot-cot,"{'text_matches': 9, 'text_match_v1': 7, 'text_...","{'text_matches': 8, 'text_match_v1': 7, 'text_...","{'text_matches': 7, 'text_match_v1': 5, 'text_...","{'text_matches': 5, 'text_match_v1': 3, 'text_...","{'text_matches': 10, 'text_match_v1': 10, 'tex..."
zero-shot-self-verification,"{'text_matches': 7, 'text_match_v1': 7, 'text_...","{'text_matches': 12, 'text_match_v1': 11, 'tex...","{'text_matches': 6, 'text_match_v1': 6, 'text_...","{'text_matches': 8, 'text_match_v1': 3, 'text_...","{'text_matches': 8, 'text_match_v1': 8, 'text_..."
zero-shot-cot,"{'text_matches': 10, 'text_match_v1': 9, 'text...","{'text_matches': 11, 'text_match_v1': 10, 'tex...","{'text_matches': 8, 'text_match_v1': 7, 'text_...","{'text_matches': 7, 'text_match_v1': 5, 'text_...","{'text_matches': 7, 'text_match_v1': 7, 'text_..."
few-shot,"{'text_matches': 9, 'text_match_v1': 9, 'text_...","{'text_matches': 9, 'text_match_v1': 9, 'text_...","{'text_matches': 4, 'text_match_v1': 4, 'text_...","{'text_matches': 6, 'text_match_v1': 5, 'text_...","{'text_matches': 10, 'text_match_v1': 10, 'tex..."
few-shot-self-verification-cot,"{'text_matches': 8, 'text_match_v1': 7, 'text_...","{'text_matches': 11, 'text_match_v1': 9, 'text...","{'text_matches': 5, 'text_match_v1': 5, 'text_...","{'text_matches': 8, 'text_match_v1': 4, 'text_...","{'text_matches': 11, 'text_match_v1': 11, 'tex..."


In [12]:
text_match_df = df.copy()
for column in text_match_df.columns:
    text_match_df[column] = text_match_df[column].apply(
        lambda x: x["text_matches"] / x["total"] if x["total"] > 0 else 0
    )
text_match_df

,ambiguity_inText,inconsistencies_inText,misaligned_terminalogy_inText,omissions_inText,structural_flaws_inText
few-shot-self-verification,0.533333,0.666667,0.533333,0.400000,0.733333
zero-shot,0.533333,0.800000,0.687500,0.466667,0.588235
zero-shot-self-verification-cot,0.500000,0.571429,0.476190,0.400000,0.476190
few-shot-cot,0.600000,0.533333,0.411765,0.333333,0.666667
zero-shot-self-verification,0.466667,0.545455,0.333333,0.444444,0.421053
zero-shot-cot,0.666667,0.578947,0.500000,0.466667,0.437500
few-shot,0.600000,0.600000,0.266667,0.400000,0.666667
few-shot-self-verification-cot,0.533333,0.611111,0.333333,0.444444,0.733333


In [13]:
text_match_df = df.copy()
for column in text_match_df.columns:
    text_match_df[column] = text_match_df[column].apply(
        lambda x: x["correct"] / x["total"] if x["total"] > 0 else 0
    )
text_match_df

,ambiguity_inText,inconsistencies_inText,misaligned_terminalogy_inText,omissions_inText,structural_flaws_inText
few-shot-self-verification,0.066667,0.066667,0.066667,0.000000,0.266667
zero-shot,0.266667,0.066667,0.125000,0.133333,0.352941
zero-shot-self-verification-cot,0.125000,0.000000,0.095238,0.000000,0.142857
few-shot-cot,0.200000,0.000000,0.058824,0.133333,0.600000
zero-shot-self-verification,0.133333,0.090909,0.055556,0.000000,0.157895
zero-shot-cot,0.266667,0.052632,0.062500,0.000000,0.312500
few-shot,0.200000,0.133333,0.066667,0.200000,0.333333
few-shot-self-verification-cot,0.133333,0.055556,0.066667,0.000000,0.333333


In [14]:
def aggregate_correct_score(row):
    total = 0
    correct = 0
    for col in row.index:
        total += row[col]["total"]
        correct += row[col]["correct"]
    return correct / total if total > 0 else 0
        
# Text Match
total_score = df.copy()
total_score.apply(aggregate_correct_score, axis=1)

few-shot-self-verification         0.093333
zero-shot                          0.192308
zero-shot-self-verification-cot    0.072165
few-shot-cot                       0.194805
zero-shot-self-verification        0.086957
zero-shot-cot                      0.135802
few-shot                           0.186667
few-shot-self-verification-cot     0.111111
dtype: float64

In [15]:
def aggregate_correct_score(row):
    total = 0
    correct = 0
    for col in row.index:
        total += row[col]["total"]
        correct += row[col]["text_matches"]
    return correct / total if total > 0 else 0
        
# Text Match
total_score = df.copy()
total_score.apply(aggregate_correct_score, axis=1)

few-shot-self-verification         0.573333
zero-shot                          0.615385
zero-shot-self-verification-cot    0.474227
few-shot-cot                       0.506494
zero-shot-self-verification        0.445652
zero-shot-cot                      0.530864
few-shot                           0.506667
few-shot-self-verification-cot     0.530864
dtype: float64

In [ ]:
def explanation_match_with_threshold(row, threshold):
    total = 0
    correct = 0

    for col in row.index:
        total += row[col]["total"]
        correct += sum([1 if score > threshold else 0 for score in row[col]["explanation_match_scores"]])
    return correct / total if total > 0 else 0
        
# Text Match
total_score = df.copy()
total_score.apply(lambda x : explanation_match_with_threshold(x, 0.5), axis=1)

few-shot-self-verification         0.586667
zero-shot                          0.679487
zero-shot-self-verification-cot    0.597938
few-shot-cot                       0.636364
zero-shot-self-verification        0.500000
zero-shot-cot                      0.592593
few-shot                           0.573333
few-shot-self-verification-cot     0.604938
dtype: float64

#### Few-shot variations

## TODO 
---
- Z ✅
- Z + COT ✅
- Z + SV ✅
- Z + COT + SV ✅
- Z + SC ✅
- Z + COT + SC ✅
---
- FS ✅⚠️
- FS + COT ✅⚠️
- FS + SV ✅⚠️
- FS + COT + SV ✅⚠️
- FS + SC ✅⚠️
- FS + COT + SC ✅⚠️
---
- Z + SV + SC (SKIP THIS FOR NOW) ✅
- Z + COT + SV + SC (SKIP THIS FOR NOW) ✅
- FS + SV + SC (SKIP THIS FOR NOW) ✅⚠️
- FS + COT + SV + SC (SKIP THIS FOR NOW) ✅⚠️
---
- **Output into a .csv**❌
- **Eventually need to repeat with different LLMs**❌